### Завдання: "Генератор щоденника виконання функцій"

**Опис завдання**:

Ви - розробник, який працює над великим проектом. Для кращого моніторингу виконання функцій ви вирішили створити декоратор, який буде записувати інформацію про кожен виклик функції. Також ви хочете мати можливість перебирати всі записи у порядку їх додавання за допомогою генератора.

**Завдання**:

1. **Створіть декоратор `@logger`, який буде:**
    - Записувати час виклику, назву функції, передані аргументи та отриманий результат.
    - Зберігати цю інформацію у якійсь внутрішній структурі (наприклад, у списку).

2. **Створіть генератор `get_logs()`, який буде:**
    - Повертати по одному запису з історії викликів кожного разу, коли його викликають.

**Приклад використання**:

```python
@logger
def multiply(a, b):
    return a * b

multiply(3, 4)
multiply(5, 6)

log = get_logs()

print(next(log))  # Повинно повернути інформацію про виклик `multiply(3, 4)`
print(next(log))  # Повинно повернути інформацію про виклик `multiply(5, 6)`
```

**Підказки**:

1. Декоратор може використовувати `datetime.now()` для отримання часу виклику.
2. Генератору, можливо, знадобиться мати доступ до внутрішньої структури декоратора, щоб перебирати записи.

## Додаткове завдання
- зробити запис логу у файл. Повторити всі вказані операції, при умові синхронізації логу у файлі
- розширити функціонал декоратору @log додавши можливість збереження типу функції (наприклад, лог помилок, лог виклику фунцкції з ворнінгом і т.п.)

In [1]:
import datetime

call_history = []

def logger(log_type='function_call'):
    def decorator(func):
        def wrapper(*args, **kwargs):
            result = None

            call_time = datetime.datetime.now()
            
            function_name = func.__name__

            arguments = {'args': args, 'kwargs': kwargs}
            
            try:
                result = func(*args, **kwargs)
                log_entry = {
                    'log_type': log_type,
                    'call_time': call_time,
                    'function_name': function_name,
                    'arguments': arguments,
                    'result': result
                }
            except Exception as e:
                log_entry = {
                    'log_type': 'error',
                    'call_time': call_time,
                    'function_name': function_name,
                    'arguments': arguments,
                    'error_message': str(e)
                }
            finally:
                call_history.append(log_entry)
            with open('function_logs.txt', 'a') as file:
                file.write(f"{log_entry}\n")
            return result

        return wrapper
    return decorator

def get_logs():
    for entry in call_history:
        yield entry

@logger(log_type='function_call')
def multiply(a, b):
    return a * b

@logger(log_type='warning')
def divide(a, b):
    if b == 0:
        raise ValueError("Division by zero is not allowed")
    return a / b

multiply(8, 6)
divide(9, 0)

log = get_logs()

for entry in log:
    print(entry)


{'log_type': 'function_call', 'call_time': datetime.datetime(2023, 10, 24, 9, 15, 27, 935523), 'function_name': 'multiply', 'arguments': {'args': (8, 6), 'kwargs': {}}, 'result': 48}
{'log_type': 'error', 'call_time': datetime.datetime(2023, 10, 24, 9, 15, 27, 935523), 'function_name': 'divide', 'arguments': {'args': (9, 0), 'kwargs': {}}, 'error_message': 'Division by zero is not allowed'}
